In [ ]:
import nltk
import numpy as np

from nltk.corpus import movie_reviews

nltk.download('punkt')
nltk.download('movie_reviews')

NUM_SAMPLES = 100
fileids = np.asarray(movie_reviews.fileids())

idx = np.concatenate([
    np.arange(NUM_SAMPLES // 2), 
    np.arange(len(fileids) - NUM_SAMPLES // 2, len(fileids))
])

print(fileids[idx])

In [5]:
from collections import Counter
from nltk.corpus import movie_reviews

reviews = []
labels = []

for fileid in fileids[idx]:
    reviews.append(movie_reviews.raw(fileid))
    labels.append(fileid.split('/')[0])
    
print(reviews[2])
print(labels[2])
print(Counter(labels))

it is movies like these that make a jaded movie viewer thankful for the invention of the timex indiglo watch . 
based on the late 1960's television show by the same name , the mod squad tells the tale of three reformed criminals under the employ of the police to go undercover . 
however , things go wrong as evidence gets stolen and they are immediately under suspicion . 
of course , the ads make it seem like so much more . 
quick cuts , cool music , claire dane's nice hair and cute outfits , car chases , stuff blowing up , and the like . 
sounds like a cool movie , does it not ? 
after the first fifteen minutes , it quickly becomes apparent that it is not . 
the mod squad is certainly a slick looking production , complete with nice hair and costumes , but that simply isn't enough . 
the film is best described as a cross between an hour-long cop show and a music video , both stretched out into the span of an hour and a half . 
and with it comes every single clich ? . 
it doesn't really 

In [15]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from typing import List

def tokenize(sentence: List[str], remove_stopwords: bool = False):
    tokens = word_tokenize(sentence)
    tokens = [token.lower() for token in tokens]
    tokens = ['<s>'] + tokens + ['<e>']
    
    if remove_stopwords:
        tokens = [token for token in tokens if not token in stopwords.words('english')]
    
    return tokens
    
tokens = [tokenize(review, remove_stopwords=True) for review in reviews]
print(tokens[:10])
y = np.asarray([0 if label == 'neg' else 1 for label in labels])
print(y[:10])

[['<s>', 'plot', ':', 'two', 'teen', 'couples', 'go', 'church', 'party', ',', 'drink', 'drive', '.', 'get', 'accident', '.', 'one', 'guys', 'dies', ',', 'girlfriend', 'continues', 'see', 'life', ',', 'nightmares', '.', "'s", 'deal', '?', 'watch', 'movie', '``', 'sorta', '``', 'find', '.', '.', '.', 'critique', ':', 'mind-fuck', 'movie', 'teen', 'generation', 'touches', 'cool', 'idea', ',', 'presents', 'bad', 'package', '.', 'makes', 'review', 'even', 'harder', 'one', 'write', ',', 'since', 'generally', 'applaud', 'films', 'attempt', 'break', 'mold', ',', 'mess', 'head', '(', 'lost', 'highway', '&', 'memento', ')', ',', 'good', 'bad', 'ways', 'making', 'types', 'films', ',', 'folks', "n't", 'snag', 'one', 'correctly', '.', 'seem', 'taken', 'pretty', 'neat', 'concept', ',', 'executed', 'terribly', '.', 'problems', 'movie', '?', 'well', ',', 'main', 'problem', "'s", 'simply', 'jumbled', '.', 'starts', '``', 'normal', '``', 'downshifts', '``', 'fantasy', '``', 'world', ',', 'audience', 'me

In [46]:
from functools import reduce

dictionary = sorted(set(reduce(lambda x, y: x + y, tokens)))
print(dictionary[:10])
encoded = [np.asarray([dictionary.index(token) for token in sentence]) for sentence in tokens]
#print(encoded[:10])
max_length = np.amax([len(sentence) for sentence in encoded])

def pad_to_length(sequence: List[int], max_length: int):
    remaining = max_length - len(sequence)
    padded = np.pad(sequence, (0, remaining), mode='constant', constant_values=len(dictionary))
    
    return np.asarray(padded)
padded = np.asarray([pad_to_length(sentence, max_length) for sentence in encoded])


['!', '#', '$', '%', '&', "'", "'80s", "'97", "'being", "'bend"]


In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Embedding, SimpleRNN

X = padded
y = np.reshape(np.asarray(y), (-1, 1))

inputs = Input((None,))
embedding = Embedding(input_dim=len(dictionary) + 1, output_dim=128, mask_zero=True)(inputs)
rnn = SimpleRNN(64)(embedding)
dense = Dense(1, activation='sigmoid')(rnn)

model = Model(inputs, dense)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X, y)

2024-05-11 23:56:00.411820: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/4 [==============>...............] - ETA: 18:43 - loss: 0.6951 - acc: 0.4844